<a href="https://colab.research.google.com/github/ishikaarora/pytorch-tutorials/blob/master/Attacks_final_original_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function, division
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.autograd import Variable
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
import random
import pandas as pd
import numpy as np
from scipy import interp
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import LabelBinarizer
plt.ion()   # interactive mode
from google.colab import drive
from torchvision.datasets import CIFAR10


##  Model classes

In [0]:
class mymodel(nn.Module):
    """Custom Model"""

    def __init__(self, in_features, hidden_dim, n_classes):
        """Custom Model Builder."""
        super(mymodel, self).__init__()
        
        self.fc1 = nn.Linear(in_features, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim//2)
        self.fc3 = nn.Linear(hidden_dim//2, n_classes)
   
  
    def forward(self, images):
        """Perform forward."""
        
        images_vec = images.view(images.size(0), -1)
        hidden_out = F.relu(self.fc1(images_vec))
        hidden_out = F.relu(self.fc2(hidden_out))
        scores = self.fc3(hidden_out)

        return scores

## Atack functions

In [0]:
# FGSM Gradient attack code
def gradient_attack(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    # perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image

### FGSM Attack

In [0]:
# FGSM Attack
def fgsm_attack(model, data, target, **kwargs):
    loss_fn = kwargs.get('loss_fn', nn.CrossEntropyLoss())
    epsilon = kwargs.get('epsilon', 0.1)
    data.requires_grad = True
    
    output = model(data)
    loss = loss_fn(output,target) 
    model.zero_grad()
    loss.backward()
    data_grad = data.grad.data
    perturbed_data = gradient_attack(data, epsilon, data_grad)

    return perturbed_data

### Transferability Attack

Load pretrained model on CIFAR-10

In [0]:
# Load the proxy model for Transfer attack
proxy_model = torch.load(root_path + "models/myproxymodel_resnet.pt", map_location=torch.device(device))
# proxy_model = torch.load("myproxymodel_resnet.pt", map_location=torch.device(device))
# We don't want to train the model, so tell PyTorch not to compute gradients
# with respect to model parameters.
for param in proxy_model.parameters():
    param.requires_grad = False

In [0]:
proxy_acc, data_size = calculate_accuracy(proxy_model, testloader)
print(proxy_acc, data_size)

Current Accuracy: 792 / 1000 = 79.2
Current Accuracy: 1585 / 2000 = 79.25
Current Accuracy: 2380 / 3000 = 79.33333333333333
Current Accuracy: 3158 / 4000 = 78.95
Current Accuracy: 3970 / 5000 = 79.4
Current Accuracy: 4767 / 6000 = 79.45
Current Accuracy: 5541 / 7000 = 79.15714285714286
Current Accuracy: 6355 / 8000 = 79.4375
Current Accuracy: 7140 / 9000 = 79.33333333333333
Current Accuracy: 7946 / 10000 = 79.46
79.46 10000


In [0]:
def transfer_attack(proxy_model, data, target, **kwargs):
    # Attack setup
    loss_fn = kwargs.get('loss_fn', nn.CrossEntropyLoss())
    epsilon = kwargs.get('epsilon', 0.1)
    data.requires_grad = True
    
    proxy_output = proxy_model(data)
    # Calculate the loss, forward pass
    loss = loss_fn(proxy_output, target)
    # Zero all existing gradients
    proxy_model.zero_grad()
    # Calculate gradients of model in backward pass
    loss.backward()
    # Collect datagrad
    data_grad = data.grad.data
    
    # Call FGSM Attack
    perturbed_data = gradient_attack(data, epsilon, data_grad)
    
    return perturbed_data

### CW Attack

In [0]:
def cw_attack(model, images, labels, **kwargs):
  cw_attack_model = torchattacks.CW(model)
  adversarial_images = cw_attack_model(images, labels)
  return adversarial_images

## Util functions 

In [0]:
def class_report(y_true, y_pred, y_score=None, average='micro'):
    print(type(y_pred),type(y_score),y_pred.shape,y_score.shape)
    if y_true.shape != y_pred.shape:
        print("Error! y_true %s is not the same shape as y_pred %s" % (
              y_true.shape,
              y_pred.shape)
        )
        return

    lb = LabelBinarizer()

    if len(y_true.shape) == 1:
        lb.fit(y_true)

    #Value counts of predictions
    labels, cnt = np.unique(
        y_pred,
        return_counts=True)
    n_classes = len(labels)
    pred_cnt = pd.Series(cnt, index=labels)

    metrics_summary = precision_recall_fscore_support(
            y_true=y_true,
            y_pred=y_pred,
            labels=labels)

    avg = list(precision_recall_fscore_support(
            y_true=y_true,
            y_pred=y_pred,
            average='weighted'))

    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support']
    class_report_df = pd.DataFrame(
        list(metrics_summary),
        index=metrics_sum_index,
        columns=labels)

    support = class_report_df.loc['support']
    total = support.sum()
    class_report_df['avg / total'] = avg[:-1] + [total]

    class_report_df = class_report_df.T
    class_report_df['pred'] = pred_cnt
    class_report_df['pred'].iloc[-1] = total

    if not (y_score is None):
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for label_it, label in enumerate(labels):
            fpr[label], tpr[label], _ = roc_curve(
                (y_true == label).astype(int),
                y_score[:, label_it])

            roc_auc[label] = auc(fpr[label], tpr[label])

        if average == 'micro':
            if n_classes <= 2:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                    lb.transform(y_true).ravel(),
                    y_score[:, 1].ravel())
            else:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                        lb.transform(y_true).ravel(),
                        y_score.ravel())

            roc_auc["avg / total"] = auc(
                fpr["avg / total"],
                tpr["avg / total"])

        elif average == 'macro':
            # First aggregate all false positive rates
            all_fpr = np.unique(np.concatenate([
                fpr[i] for i in labels]
            ))

            # Then interpolate all ROC curves at this points
            mean_tpr = np.zeros_like(all_fpr)
            for i in labels:
                mean_tpr += interp(all_fpr, fpr[i], tpr[i])

            # Finally average it and compute AUC
            mean_tpr /= n_classes

            fpr["macro"] = all_fpr
            tpr["macro"] = mean_tpr

            roc_auc["avg / total"] = auc(fpr["macro"], tpr["macro"])

        class_report_df['AUC'] = pd.Series(roc_auc)

    return class_report_df

In [0]:
# Only for FGSM and transfer attacks
def create_adversarial(model, images, labels, attack_type=None, **kwargs):
    if attack_type is not None:
        if attack_type == "fgsm":
            perturbed_images = fgsm_attack(model, images, labels, **kwargs)
        elif attack_type == "transfer":
            perturbed_images = transfer_attack(proxy_model, images, labels, **kwargs)
        else:
            perturbed_images = cw_attack(model, images, labels, **kwargs)
    else:    
      idx = random.uniform(0,1)

      if idx < 0.5:
          perturbed_images = fgsm_attack(model, images, labels, **kwargs)
      else:
          perturbed_images = transfer_attack(proxy_model, images, labels, **kwargs)

    return perturbed_images

In [0]:
def attack_accuracy(model, loader, attack_type=None, print_freq=1000, epsilon=0.1, run_attack=False):
    model.eval()
    total = 0
    correct = 0
    show_im = True
    all_labels = []
    all_preds = []
    all_probs = []
    #batch_count = 0 

    for data in loader:
        #batch_count+=1
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        
        if run_attack:
          kwargs = {'loss_fn': nn.CrossEntropyLoss(), 'epsilon': epsilon}     
          images = create_adversarial(model, images, labels, attack_type=attack_type, **kwargs)

        with torch.no_grad():
            outputs = model(images)
            out_prob = F.softmax(outputs, dim=1)

        _,predicted = torch.max(outputs.data,1)
        total+=labels.size(0)
        correct+=(predicted==labels).sum().item()

        all_labels.append(labels.cpu().numpy())
        all_preds.append(predicted.cpu().numpy())
        all_probs.append(out_prob.cpu().numpy())

        if total % print_freq == 0:
            print("Current Accuracy: {} / {} = {}".format(correct, total, correct/total*100))
        #if batch_count>=test_num_batches:
        #    break

    
    all_labels = np.concatenate(all_labels)
    all_preds = np.concatenate(all_preds)
    all_probs = np.concatenate(all_probs)
    print(all_labels.shape,all_preds.shape,all_probs.shape)
    report_with_auc = class_report(
                            y_true = all_labels,
                            y_pred = all_preds,
                            y_score= all_probs)
           
    return (correct/total)*100, total, report_with_auc

## Constants

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
root_path = 'gdrive/My Drive/Colab Notebooks/Adversarial/'

In [0]:
#mounting drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


## Load pre trained model

In [0]:
model = torch.load(root_path+"models/mymodel.pt", map_location=torch.device(device))
for param in model.parameters():
    param.requires_grad = False

## Load pre trained defense model

In [0]:
# distil_model1 = torch.load(root_path+"models/my_distil_model1.pt", map_location=torch.device(device))
# for param in distil_model1.parameters():
#     param.requires_grad = False

## Preparing test data sets

### Load labels

In [0]:
labels = torch.load(root_path+"samples/all_labels.pt", map_location=torch.device(device))

In [0]:
labels_1200 = labels[0:1200]
labels_1200_3099 = labels[1200:3100]
labels_3100_4999= labels[3100:5000]

### Load FGSM

In [0]:
fgsm_images = torch.load(root_path+"samples/fgsm_images_010_1200-3099.pt", map_location=torch.device(device))
fgsm_dataset = TensorDataset(fgsm_images, labels_1200_3099)
fgsm_loader = DataLoader(fgsm_dataset, batch_size= 4, shuffle=False)

In [0]:
fgsm_images1 = torch.load(root_path+"samples/fgsm_images_015_1200-3099.pt", map_location=torch.device(device))
fgsm_dataset1 = TensorDataset(fgsm_images1, labels_1200_3099)
fgsm_loader1 = DataLoader(fgsm_dataset1, batch_size= 4, shuffle=False)

### Load Transfer Attack Samples

In [0]:
transfer_images = torch.load(root_path+"samples/transfer_images_010_3100-4999.pt", map_location=torch.device(device))
transfer_dataset = TensorDataset(transfer_images, labels_3100_4999)
transfer_loader = DataLoader(transfer_dataset, batch_size= 4, shuffle=False)

In [0]:
transfer_images1 = torch.load(root_path+"samples/transfer_images_015_3100-4999.pt", map_location=torch.device(device))
transfer_dataset1 = TensorDataset(transfer_images1, labels_3100_4999)
transfer_loader1 = DataLoader(transfer_dataset1, batch_size= 4, shuffle=False)

### Load CW data

In [0]:
cw_images = torch.load(root_path+"samples/cw_images.pt", map_location=torch.device(device))
cw_labels = torch.load(root_path+"samples/cw_labels.pt", map_location=torch.device(device))
cw_dataset = TensorDataset(cw_images, cw_labels)
cw_loader = DataLoader(cw_dataset, batch_size= 4, shuffle=False)

### Load Mixture Data set

Mixed with 1st 1200 CW, next 1900 FGSM, next 1900 Transfer (eps=0.10)


In [0]:
mixed_images = torch.cat([cw_images,fgsm_images,transfer_images],dim=0)
# mixed_labels = torch.cat([labels_1200,labels_3100_4999,cw_labels],dim=0)
mixed_labels = labels[0:5000]
mixed_dataset = TensorDataset(mixed_images, mixed_labels)
mixed_loader = DataLoader(mixed_dataset, batch_size= 4, shuffle=False)

In [0]:
mixed_images1 = torch.cat([cw_images,fgsm_images1,transfer_images1],dim=0)
# mixed_labels = torch.cat([labels_1200,labels_3100_4999,cw_labels],dim=0)
mixed_labels = labels[0:5000]
mixed_dataset1 = TensorDataset(mixed_images1, mixed_labels)
mixed_loader1 = DataLoader(mixed_dataset1, batch_size= 4, shuffle=False)

### Load normal samples

In [0]:
cifar10_mean = [0.49139968 , 0.48215841 , 0.44653091]
cifar10_std = [0.24703223,  0.24348513 , 0.26158784]

#Define transformations for the test set
transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(cifar10_mean,cifar10_std)
])

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform=transform_test)

testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

0it [00:00, ?it/s]

170500096it [00:02, 69419178.56it/s]                               


Extracting ./data/cifar-10-python.tar.gz to ./data


## Test : Original Model + Normal Samples + No Detector

In [0]:
normal_sample_acc, normal_sample_data_size,normal_sample_report_with_auc = attack_accuracy(model, testloader, print_freq=1000)

170500096it [00:30, 17529495.74it/s]                               

Current Accuracy: 904 / 1000 = 90.4
Current Accuracy: 1808 / 2000 = 90.4
Current Accuracy: 2707 / 3000 = 90.23333333333333
Current Accuracy: 3603 / 4000 = 90.075
Current Accuracy: 4510 / 5000 = 90.2
Current Accuracy: 5412 / 6000 = 90.2
Current Accuracy: 6312 / 7000 = 90.17142857142856
Current Accuracy: 7211 / 8000 = 90.1375
Current Accuracy: 8126 / 9000 = 90.28888888888889
Current Accuracy: 9036 / 10000 = 90.36
(10000,) (10000,) (10000, 10)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> (10000,) (10000, 10)


In [0]:
normal_sample_report_with_auc

,precision,recall,f1-score,support,pred,AUC
0,0.911824,0.9100,0.910911,1000.0,998.0,0.996723
1,0.942214,0.9620,0.952004,1000.0,1021.0,0.998519
2,0.894737,0.8840,0.889336,1000.0,988.0,0.991217
3,0.828723,0.7790,0.803093,1000.0,940.0,0.985653
4,0.888554,0.8850,0.886774,1000.0,996.0,0.993307
5,0.849356,0.8570,0.853161,1000.0,1009.0,0.990342
6,0.919765,0.9400,0.929773,1000.0,1022.0,0.997743
7,0.918707,0.9380,0.928253,1000.0,1021.0,0.996154
8,0.935771,0.9470,0.941352,1000.0,1012.0,0.998377
9,0.940584,0.9340,0.937280,1000.0,993.0,0.997523


## Test: Original Model + CW attack samples + No detector

In [0]:
cw_sample_acc, cw_sample_data_size,cw_sample_report_with_auc = attack_accuracy(model, cw_loader, print_freq=100)

Current Accuracy: 30 / 100 = 30.0
Current Accuracy: 56 / 200 = 28.000000000000004
Current Accuracy: 77 / 300 = 25.666666666666664
Current Accuracy: 105 / 400 = 26.25
Current Accuracy: 137 / 500 = 27.400000000000002
Current Accuracy: 165 / 600 = 27.500000000000004
Current Accuracy: 191 / 700 = 27.285714285714285
Current Accuracy: 216 / 800 = 27.0
Current Accuracy: 244 / 900 = 27.111111111111114
Current Accuracy: 275 / 1000 = 27.500000000000004
Current Accuracy: 305 / 1100 = 27.727272727272727
Current Accuracy: 329 / 1200 = 27.416666666666668


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
cw_sample_report_with_auc

,precision,recall,f1-score,support,pred,AUC
0,0.266904,0.581395,0.365854,129.0,281.0,0.779225
2,0.181614,0.686441,0.287234,118.0,446.0,0.420008
3,0.294118,0.393701,0.336700,127.0,170.0,0.502653
4,0.247573,0.490385,0.329032,104.0,206.0,0.487024
5,0.714286,0.150000,0.247934,100.0,21.0,0.440482
6,0.666667,0.226950,0.338624,141.0,48.0,0.570631
7,0.923077,0.102564,0.184615,117.0,13.0,0.408907
8,0.866667,0.103175,0.184397,126.0,15.0,0.397683
avg / total,0.417992,0.274167,0.229535,962.0,962.0,0.677239


## Attacks (Epsilon=0.10)

### Test : Original Model + FGSM + No Detector![alt text](https://)

In [0]:
fgsm_acc, data_size, fgsm_report_with_auc = attack_accuracy(model, testloader, attack_type="fgsm", print_freq=1000, epsilon=0.10, run_attack=True)

Current Accuracy: 229 / 1000 = 22.900000000000002
Current Accuracy: 471 / 2000 = 23.549999999999997
Current Accuracy: 719 / 3000 = 23.966666666666665
Current Accuracy: 963 / 4000 = 24.075
Current Accuracy: 1211 / 5000 = 24.22
Current Accuracy: 1447 / 6000 = 24.116666666666667
Current Accuracy: 1699 / 7000 = 24.271428571428572
Current Accuracy: 1955 / 8000 = 24.4375
Current Accuracy: 2205 / 9000 = 24.5
Current Accuracy: 2446 / 10000 = 24.46
(10000,) (10000,) (10000, 10)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> (10000,) (10000, 10)


In [0]:
fgsm_report_with_auc

,precision,recall,f1-score,support,pred,AUC
0,0.722343,0.3330,0.455852,1000.0,461.0,0.902943
1,0.790576,0.1510,0.253568,1000.0,191.0,0.877515
2,0.298165,0.4550,0.360253,1000.0,1526.0,0.790862
3,0.137340,0.9210,0.239035,1000.0,6706.0,0.794989
4,0.419003,0.2690,0.327649,1000.0,642.0,0.790920
5,0.530612,0.1560,0.241113,1000.0,294.0,0.782839
6,1.000000,0.0110,0.021761,1000.0,11.0,0.733616
7,1.000000,0.0020,0.003992,1000.0,2.0,0.713179
8,0.887324,0.1260,0.220665,1000.0,142.0,0.907200
9,0.880000,0.0220,0.042927,1000.0,25.0,0.792264


### Test: Original Model + Transfer Attack Samples + No Detector

In [0]:
transfer_sample_acc, transfer_sample_data_size,transfer_sample_report_with_auc = attack_accuracy(model, testloader, attack_type="transfer", 
                                                                                                 print_freq=1000, epsilon=0.10, run_attack=True)

Current Accuracy: 258 / 1000 = 25.8
Current Accuracy: 558 / 2000 = 27.900000000000002
Current Accuracy: 847 / 3000 = 28.233333333333334
Current Accuracy: 1122 / 4000 = 28.050000000000004
Current Accuracy: 1423 / 5000 = 28.46
Current Accuracy: 1690 / 6000 = 28.166666666666668
Current Accuracy: 1993 / 7000 = 28.471428571428568
Current Accuracy: 2288 / 8000 = 28.599999999999998
Current Accuracy: 2578 / 9000 = 28.644444444444446
Current Accuracy: 2866 / 10000 = 28.660000000000004
(10000,) (10000,) (10000, 10)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> (10000,) (10000, 10)


In [0]:
transfer_sample_report_with_auc

,precision,recall,f1-score,support,pred,AUC
0,0.728333,0.4370,0.546250,1000.0,600.0,0.917041
1,0.840000,0.2940,0.435556,1000.0,350.0,0.915872
2,0.394664,0.4290,0.411116,1000.0,1087.0,0.819131
3,0.142985,0.9570,0.248798,1000.0,6693.0,0.833112
4,0.415631,0.2340,0.299424,1000.0,563.0,0.802960
5,0.591146,0.2270,0.328035,1000.0,384.0,0.811284
6,0.833333,0.0100,0.019763,1000.0,12.0,0.753947
7,1.000000,0.0040,0.007968,1000.0,4.0,0.751940
8,0.888446,0.2230,0.356515,1000.0,251.0,0.921131
9,0.910714,0.0510,0.096591,1000.0,56.0,0.820363


### Test: Normal Model + FGSM+Transfer Attack + No detector

In [0]:
fgsm_transfer_sample_acc, fgsm_transfer_sample_data_size,fgsm_transfer_sample_report_with_auc = attack_accuracy(model, testloader, 
                                                                                                 print_freq=1000, epsilon=0.10, run_attack=True)

Current Accuracy: 253 / 1000 = 25.3
Current Accuracy: 521 / 2000 = 26.05
Current Accuracy: 790 / 3000 = 26.333333333333332
Current Accuracy: 1048 / 4000 = 26.200000000000003
Current Accuracy: 1316 / 5000 = 26.32
Current Accuracy: 1574 / 6000 = 26.23333333333333
Current Accuracy: 1853 / 7000 = 26.471428571428575
Current Accuracy: 2134 / 8000 = 26.674999999999997
Current Accuracy: 2409 / 9000 = 26.766666666666666
Current Accuracy: 2672 / 10000 = 26.72
(10000,) (10000,) (10000, 10)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> (10000,) (10000, 10)


In [0]:
fgsm_transfer_sample_report_with_auc

,precision,recall,f1-score,support,pred,AUC
0,0.717996,0.3870,0.502924,1000.0,539.0,0.910097
1,0.821429,0.2300,0.359375,1000.0,280.0,0.895854
2,0.346426,0.4410,0.388033,1000.0,1273.0,0.802034
3,0.140301,0.9410,0.244194,1000.0,6707.0,0.816342
4,0.414594,0.2500,0.311915,1000.0,603.0,0.795543
5,0.569322,0.1930,0.288275,1000.0,339.0,0.797766
6,0.916667,0.0110,0.021739,1000.0,12.0,0.739059
7,1.000000,0.0030,0.005982,1000.0,3.0,0.732764
8,0.883249,0.1740,0.290727,1000.0,197.0,0.913795
9,0.893617,0.0420,0.080229,1000.0,47.0,0.806525


### Test: Original Model + Mixed attack samples + No detector

In [0]:
mixed_sample_acc, mixed_sample_data_size,mixed_sample_report_with_auc = attack_accuracy(model, mixed_loader, print_freq=1000)

Current Accuracy: 275 / 1000 = 27.500000000000004
Current Accuracy: 402 / 2000 = 20.1
Current Accuracy: 505 / 3000 = 16.833333333333332
Current Accuracy: 592 / 4000 = 14.799999999999999
Current Accuracy: 690 / 5000 = 13.8
(5000,) (5000,) (5000, 10)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> (5000,) (5000, 10)


In [0]:
mixed_sample_report_with_auc

,precision,recall,f1-score,support,pred,AUC
0,0.186047,0.180328,0.183143,488.0,473.0,0.562757
1,0.133929,0.029703,0.048622,505.0,112.0,0.527517
2,0.135983,0.253906,0.177112,512.0,956.0,0.568024
3,0.106771,0.577465,0.180220,497.0,2688.0,0.540488
4,0.181208,0.159763,0.169811,507.0,447.0,0.568457
5,0.173333,0.053279,0.081505,488.0,150.0,0.573587
6,0.581818,0.065173,0.117216,491.0,55.0,0.608767
7,0.923077,0.024242,0.047244,495.0,13.0,0.590013
8,0.206522,0.037698,0.063758,504.0,92.0,0.577197
9,0.000000,0.000000,0.000000,513.0,14.0,0.540157


In [0]:
mixed_sample_acc, mixed_sample_data_size,mixed_sample_report_with_auc = attack_accuracy(model, mixed_loader, print_freq=100)

Current Accuracy: 22 / 100 = 22.0
Current Accuracy: 45 / 200 = 22.5
Current Accuracy: 73 / 300 = 24.333333333333336
Current Accuracy: 93 / 400 = 23.25
Current Accuracy: 116 / 500 = 23.200000000000003
Current Accuracy: 133 / 600 = 22.166666666666668
Current Accuracy: 154 / 700 = 22.0
Current Accuracy: 175 / 800 = 21.875
Current Accuracy: 204 / 900 = 22.666666666666664
Current Accuracy: 228 / 1000 = 22.8
Current Accuracy: 257 / 1100 = 23.363636363636363
Current Accuracy: 281 / 1200 = 23.416666666666668
Current Accuracy: 288 / 1300 = 22.153846153846153
Current Accuracy: 301 / 1400 = 21.5
Current Accuracy: 308 / 1500 = 20.533333333333335
Current Accuracy: 314 / 1600 = 19.625
Current Accuracy: 321 / 1700 = 18.88235294117647
Current Accuracy: 331 / 1800 = 18.38888888888889
Current Accuracy: 341 / 1900 = 17.94736842105263
Current Accuracy: 350 / 2000 = 17.5
Current Accuracy: 360 / 2100 = 17.142857142857142
Current Accuracy: 369 / 2200 = 16.772727272727273
Current Accuracy: 377 / 2300 = 16.391

In [0]:
mixed_sample_report_with_auc

,precision,recall,f1-score,support,pred,AUC
0,0.268966,0.266515,0.267735,439.0,435.0,0.659547
1,0.238636,0.048724,0.080925,431.0,88.0,0.601527
2,0.176960,0.353081,0.235759,422.0,842.0,0.629982
3,0.129490,0.656180,0.216296,445.0,2255.0,0.623612
4,0.227979,0.217284,0.222503,405.0,386.0,0.639576
5,0.281481,0.097938,0.145315,388.0,135.0,0.628404
6,0.634615,0.072210,0.129666,457.0,52.0,0.679028
7,0.923077,0.027842,0.054054,431.0,13.0,0.643694
8,0.417722,0.076744,0.129666,430.0,79.0,0.700505
9,0.266667,0.008850,0.017131,452.0,15.0,0.611591


## Attacks (Epsilon=0.15)

### Test : Original Model + FGSM + No Detector! (eps=0.15)

In [0]:
fgsm_acc1, data_size, fgsm_report_with_auc1 = attack_accuracy(model, testloader, attack_type="fgsm", print_freq=1000, epsilon=0.15, run_attack=True)

Current Accuracy: 161 / 1000 = 16.1
Current Accuracy: 334 / 2000 = 16.7
Current Accuracy: 515 / 3000 = 17.166666666666668
Current Accuracy: 690 / 4000 = 17.25
Current Accuracy: 857 / 5000 = 17.14
Current Accuracy: 1020 / 6000 = 17.0
Current Accuracy: 1184 / 7000 = 16.914285714285715
Current Accuracy: 1374 / 8000 = 17.175
Current Accuracy: 1552 / 9000 = 17.244444444444447
Current Accuracy: 1718 / 10000 = 17.18
(10000,) (10000,) (10000, 10)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> (10000,) (10000, 10)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
fgsm_report_with_auc1

,precision,recall,f1-score,support,pred,AUC
0,0.735751,0.1420,0.238055,1000.0,193.0,0.869045
1,0.850000,0.0170,0.033333,1000.0,20.0,0.782978
2,0.188291,0.5950,0.286058,1000.0,3160.0,0.742476
3,0.133302,0.8470,0.230351,1000.0,6354.0,0.714751
4,0.385892,0.0930,0.149879,1000.0,241.0,0.745703
5,0.714286,0.0150,0.029383,1000.0,21.0,0.727394
8,0.818182,0.0090,0.017804,1000.0,11.0,0.576015
avg / total,0.382570,0.1718,0.098486,7000.0,7000.0,0.618208


### Test: Original Model + Transfer Attack Samples + No Detector (eps=0.15)

In [0]:
transfer_sample_acc1, transfer_sample_data_size,transfer_sample_report_with_auc1 = attack_accuracy(model, testloader, attack_type="transfer", 
                                                                                                 print_freq=1000, epsilon=0.15, run_attack=True)

Current Accuracy: 182 / 1000 = 18.2
Current Accuracy: 377 / 2000 = 18.85
Current Accuracy: 563 / 3000 = 18.76666666666667
Current Accuracy: 759 / 4000 = 18.975
Current Accuracy: 937 / 5000 = 18.740000000000002
Current Accuracy: 1112 / 6000 = 18.53333333333333
Current Accuracy: 1295 / 7000 = 18.5
Current Accuracy: 1509 / 8000 = 18.862499999999997
Current Accuracy: 1703 / 9000 = 18.922222222222224
Current Accuracy: 1885 / 10000 = 18.85
(10000,) (10000,) (10000, 10)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> (10000,) (10000, 10)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
transfer_sample_report_with_auc1

,precision,recall,f1-score,support,pred,AUC
0,0.707602,0.2420,0.360656,1000.0,342.0,0.879715
1,0.840000,0.0420,0.080000,1000.0,50.0,0.824931
2,0.218015,0.5470,0.311770,1000.0,2509.0,0.768159
3,0.132025,0.8880,0.229873,1000.0,6726.0,0.746495
4,0.368932,0.1140,0.174179,1000.0,309.0,0.745296
5,0.750000,0.0300,0.057692,1000.0,40.0,0.747316
8,0.916667,0.0220,0.042969,1000.0,24.0,0.576040
avg / total,0.393324,0.1885,0.125714,7000.0,7000.0,0.642671


### Test: Original Model + FGSM+Transfer Attack Samples + No Detector (eps=0.15)

In [0]:
fgsm_transfer_sample_acc1, fgsm_transfer_sample_data_size1,fgsm_transfer_sample_report_with_auc1 = attack_accuracy(model, testloader, 
                                                                                                 print_freq=1000, epsilon=0.15, run_attack=True)

Current Accuracy: 171 / 1000 = 17.1
Current Accuracy: 357 / 2000 = 17.849999999999998
Current Accuracy: 541 / 3000 = 18.033333333333335
Current Accuracy: 725 / 4000 = 18.125
Current Accuracy: 899 / 5000 = 17.98
Current Accuracy: 1072 / 6000 = 17.866666666666667
Current Accuracy: 1252 / 7000 = 17.885714285714286
Current Accuracy: 1456 / 8000 = 18.2
Current Accuracy: 1637 / 9000 = 18.188888888888886
Current Accuracy: 1810 / 10000 = 18.099999999999998
(10000,) (10000,) (10000, 10)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> (10000,) (10000, 10)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
fgsm_transfer_sample_report_with_auc1

,precision,recall,f1-score,support,pred,AUC
0,0.719101,0.192,0.303078,1000.0,267.0,0.875069
1,0.833333,0.030,0.057915,1000.0,36.0,0.803839
2,0.203521,0.578,0.301042,1000.0,2840.0,0.758690
3,0.133497,0.872,0.231545,1000.0,6532.0,0.734421
4,0.371429,0.104,0.162500,1000.0,280.0,0.744092
5,0.700000,0.021,0.040777,1000.0,30.0,0.734126
8,0.866667,0.013,0.025616,1000.0,15.0,0.576300
avg / total,0.382755,0.181,0.112247,7000.0,7000.0,0.630336


### Test: Original Model + Mixed attack samples + No detector (eps=0.15)

In [0]:
mixed_sample_acc1, mixed_sample_data_size,mixed_sample_report_with_auc1 = attack_accuracy(model, mixed_loader1, print_freq=1000)

Current Accuracy: 275 / 1000 = 27.500000000000004
Current Accuracy: 402 / 2000 = 20.1
Current Accuracy: 521 / 3000 = 17.366666666666667
Current Accuracy: 608 / 4000 = 15.2
Current Accuracy: 709 / 5000 = 14.180000000000001
(5000,) (5000,) (5000, 10)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> (5000,) (5000, 10)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
mixed_sample_report_with_auc1

,precision,recall,f1-score,support,pred,AUC
0,0.216495,0.172131,0.191781,488.0,388.0,0.574669
1,0.153846,0.003960,0.007722,505.0,13.0,0.525126
2,0.132720,0.394531,0.198623,512.0,1522.0,0.580660
3,0.107832,0.573441,0.181529,497.0,2643.0,0.556542
4,0.192547,0.122288,0.149578,507.0,322.0,0.578001
5,0.548387,0.034836,0.065511,488.0,31.0,0.575981
6,0.666667,0.065173,0.118738,491.0,48.0,0.595992
7,0.923077,0.024242,0.047244,495.0,13.0,0.590431
8,0.650000,0.025794,0.049618,504.0,20.0,0.580904
avg / total,0.356395,0.141800,0.100781,4487.0,4487.0,0.552841
